In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Generate synthetic data
np.random.seed(42)
data = {
    "Moneyness": np.random.uniform(0.8, 1.2, 1000),
    "Interest_Rates": np.random.uniform(1, 5, 1000),
    "Par_Rates": np.random.uniform(2, 6, 1000),
    "Loan_Amount": np.random.randint(10000, 50000, 1000),
    "Loan_Type": np.random.choice(["Personal", "Mortgage", "Auto"], 1000),
    "Duration": np.random.randint(1, 30, 1000),
    "Credit_Score": np.random.randint(600, 800, 1000),
    "Price": np.random.uniform(1000, 5000, 1000)  # Target variable
}
df = pd.DataFrame(data)

# Preprocessing
categorical_features = ['Loan_Type']
numeric_features = ["Moneyness", "Interest_Rates", "Par_Rates", "Loan_Amount", "Duration", "Credit_Score"]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Split data
X = df.drop("Price", axis=1)
y = df["Price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Define model
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

# Train model
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
print(f"R^2 Score: {r2_score(y_test, y_pred)}")
print(f"RMSE: {mean_squared_error(y_test, y_pred, squared=False)}")

# Interpretability
coefficients = model.named_steps['regressor'].coef_
features = numeric_features + list(model.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(categorical_features))
interpretability_df = pd.DataFrame({'Feature': features, 'Coefficient': coefficients})

print(interpretability_df)

R^2 Score: -0.05433107841236717
RMSE: 1208.8093589092114
              Feature  Coefficient
0           Moneyness   538.567963
1      Interest_Rates    28.604463
2           Par_Rates    55.717370
3         Loan_Amount    -0.004516
4            Duration    -2.440916
5        Credit_Score     0.764988
6      Loan_Type_Auto   138.387246
7  Loan_Type_Mortgage   -54.620558
8  Loan_Type_Personal   -83.766688


In [5]:
from openpyxl.styles import Font, Color, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill


# Define Excel writer and save DataFrame
excel_path = 'interpretability_analysis_with_styles.xlsx'
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    interpretability_df.to_excel(writer, sheet_name='Interpretability', index=False)
    wb = writer.book
    ws = wb['Interpretability']

    # Apply styles to header row
    for cell in ws["1:1"]:
        cell.font = Font(bold=True, color="FFFFFF")
        cell.fill = PatternFill(start_color="4F81BD", end_color="4F81BD", fill_type="solid")
        cell.alignment = Alignment(horizontal="center")


In [6]:
import matplotlib.pyplot as plt
import seaborn as sns

# Generate a plot
plt.figure(figsize=(10, 6))
sns.barplot(x='Coefficient', y='Feature', data=interpretability_df.sort_values(by='Coefficient', ascending=False))
plt.title('Feature Importance')
plt.tight_layout()

# Save the plot as an image
plot_path = 'feature_importance.png'
plt.savefig(plot_path)
plt.close()


In [7]:
from openpyxl.drawing.image import Image
from openpyxl.styles import PatternFill
from openpyxl import load_workbook


# Reload the workbook and select the sheet
wb = load_workbook(excel_path)
ws = wb.create_sheet('Plots')

# Embed the image
img = Image(plot_path)
ws.add_image(img, 'A1')

# Example of adding styled text to the Excel file
ws['A10'] = 'Feature Importance Analysis'
ws['A10'].font = Font(bold=True, size=14, color="0070C0")
ws['A10'].alignment = Alignment(horizontal="center")

# Save the workbook with the embedded image and styled text
wb.save(excel_path)
